# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [4]:
ruta_proyecto = 'C:/Users/User/Desktop/DS4B/03_MACHINE_LEARNING/08_CASOS/01_leadScoring'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [5]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

4.- Separar la target.

In [7]:
target = num[['compra']].copy().reset_index(drop=True)

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [15]:
var_ohe = cat.columns.to_list()

#### Instanciar

In [16]:
ohe = OneHotEncoder(sparse_output= False, handle_unknown='ignore')

#### Entrenar y aplicar

In [17]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [18]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

## TRANSFORMACIÓN DE NUMÉRICAS

## UNIFICAR DATASETS TRANSFORMADOS

In [22]:
df = pd.concat([cat_ohe, num.reset_index()], axis=1).set_index('id')

## REESCALAR VARIABLES

### Con Min-Max

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4812 entries, 660737 to 579533
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4812 non-null   float64
 1   origen_Landing Page Submission            4812 non-null   float64
 2   origen_Lead Add Form                      4812 non-null   float64
 3   origen_OTROS                              4812 non-null   float64
 4   fuente_Chat                               4812 non-null   float64
 5   fuente_Direct Traffic                     4812 non-null   float64
 6   fuente_Google                             4812 non-null   float64
 7   fuente_OTROS                              4812 non-null   float64
 8   fuente_Organic Search                     4812 non-null   float64
 9   fuente_Reference                          4812 non-null   float64
 10  ult_actividad_Chat Conversation   

#### Variables a reescalar con Min-Max

In [31]:
num.iloc[:, 1:].columns

Index(['visitas_total', 'tiempo_en_site_total', 'paginas_vistas_visita',
       'score_actividad', 'score_perfil'],
      dtype='object')

In [32]:
var_mms = num.iloc[:, 1:].columns

#### Instanciar

In [34]:
var_mms

Index(['visitas_total', 'tiempo_en_site_total', 'paginas_vistas_visita',
       'score_actividad', 'score_perfil'],
      dtype='object')

In [33]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [35]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [36]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

## UNIFICAR DATASETS REESCALADOS

### Unir todos los dataframes en el tablón analítico

In [45]:
df_tablon = pd.concat([cat_ohe, df_mms, target, df.reset_index().id], axis=1).set_index('id')

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [46]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [47]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)